In [1]:
import math
import statistics
import numpy
import csv
import scipy as sp
from scipy import interpolate
from scipy.stats import poisson
from scipy.stats import norm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import seaborn as sns
import pylab
from scipy.stats import kendalltau
import matplotlib
from matplotlib.ticker import FuncFormatter, MultipleLocator
import matplotlib.cm as cm
from matplotlib.widgets import Slider
from matplotlib.ticker import FuncFormatter
import matplotlib.ticker as ticker
import plotly.plotly as py
import cufflinks as cf
import plotly.graph_objs as go
from scipy.stats import invgamma
py.sign_in('PythonAPI', 'ubpiol2cve')

In [7]:
global_users = pd.read_csv('../../fb2015/users.csv', header=None)#.sample(frac=0.01)
global_users.columns = ['userId', 'registerCountry', 'signupTime']
global_conv = pd.read_csv('../../fb2015/conversions.csv', header=None)#.sample(frac=0.01)
global_conv.columns = ['userId', 'itemId', 'price', 'quantity', 'timestamp']
global_items = pd.read_csv('../../fb2015/items.csv', header=None)#.sample(frac=0.01)
global_items.columns = ['itemId', 'style', 'personality', 'color', 'theme', 'price', 'category']
global_users_ads = pd.read_csv('../../fb2015/users_ads.csv', header=None)#.sample(frac=0.01)
global_users_ads.columns = ['userId', 'utmSource', 'utmCampaign', 'utmMedium', 'utmTerm', 'utmContent']

Obtains data from files.

In [2]:
def drop_nulls(table, columns):
    table = table.dropna(subset=columns)
    for col in columns:
        table = table[~table[col].isin(['None'])]
    return table

Drops NaN, None values from given data.

In [ ]:
def number_of_new_users_per_month():
    users = drop_nulls(global_users, ['signupTime', 'userId'])

    users['month'] = pd.DatetimeIndex(users['signupTime']).month
    users['year'] = pd.DatetimeIndex(users['signupTime']).year

    grouped = users.groupby(['year', 'month'])['userId'].count()
    sortedd = grouped.sort_index()

    ax = sortedd.plot(kind='bar')

    ax.set_xlabel('Month')
    ax.set_ylabel('Number of new users')
    ax.set_title("Number of new users per month.", fontsize=18)

    plt.xticks(rotation=70)
    plt.tight_layout()
    plt.show()

Displays number of newly registered users per month.

In [ ]:
def number_of_conversions_per_month():
    conv = drop_nulls(global_conv, ['timestamp'])

    conv['month'] = pd.DatetimeIndex(conv['timestamp']).month
    conv['year'] = pd.DatetimeIndex(conv['timestamp']).year

    grouped = conv.groupby(['year', 'month']).count()
    sortedd = grouped.sort_index()

    ax = sortedd.plot(legend=None)
    ax.set_xlabel('Month')
    ax.set_ylabel('Number of conversions')
    ax.set_title("Number of conversions per month.", fontsize=18)

    plt.xticks(rotation=70)
    plt.tight_layout()

    plt.show()

Displays number of conversions per month.

In [ ]:
def number_of_items_purchased_per_month():
    conv = drop_nulls(global_conv, ['timestamp', 'quantity'])

    conv['month'] = pd.DatetimeIndex(conv['timestamp']).month
    conv['year'] = pd.DatetimeIndex(conv['timestamp']).year

    grouped = conv.groupby(['year', 'month'])['quantity'].sum()
    sortedd = grouped.sort_index()

    ax = sortedd.plot(kind='bar')
    ax.set_xlabel('Month')
    ax.set_ylabel('Number of items')
    ax.set_title("Number of items purchased per month.", fontsize=18)

    plt.xticks(rotation=70)
    plt.tight_layout()

    plt.show()

Displays number of items purchased per month.

In [61]:
def revenue_per_month():
    conv = drop_nulls(global_conv, ['timestamp', 'price'])

    conv['month'] = pd.DatetimeIndex(conv['timestamp']).month
    conv['year'] = pd.DatetimeIndex(conv['timestamp']).year
    grouped = conv.groupby(['year', 'month'])['price'].sum()
    sortedd = grouped.sort_index()

    ax = sortedd.plot() 
    ax.set_xlabel('Month')
    ax.set_ylabel('Revenue')
    ax.set_title("Revenue per month.", fontsize=18)

    plt.xticks(rotation=70)
    plt.tight_layout()

    plt.show()

Displays income per month.

In [6]:
def number_of_items_purchased_per_user_in_the_first_week_after_registration_histogram():
    #does not include users without purchase in the first week after registration
    users = drop_nulls(global_users, ['userId', 'signupTime'])
    conv = drop_nulls(global_conv, ['userId', 'quantity', 'timestamp'])

    users['signupTime'] =  pd.DatetimeIndex(users['signupTime'])
    users['week_after'] = pd.DatetimeIndex(users['signupTime'])+pd.Timedelta(days=7)
    conv['timestamp'] = pd.DatetimeIndex(conv['timestamp'])

    joined = pd.merge(users, conv, left_on='userId', right_on='userId', how='inner')
    grouped = joined[joined['timestamp'] <= joined['week_after']].groupby('userId')['quantity'].sum()

    bins = range(1, max(grouped.values)+2)
    #bins = bins[0::10]

    fig = plt.figure()
    ax = fig.add_subplot(111)

    ax.hist(grouped.values, bins=bins, align="left", facecolor='green', label='Histogram of the data') 

    ax.set_xlabel('Bins')
    ax.set_ylabel('Quantity')
    ax.set_title("Histogram: number of items purchased per user in the first week after registration (does not include users without purchase).", fontsize=18)

    plt.show()

/home/gaba/anaconda3/lib/python3.5/site-packages/numpy/lib/arraysetops.py:379: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



Displays number of items purchased during the first week after signing in grouped into bins.

In [4]:
def number_of_items_purchased_from_particular_category_grouped_by_country(category):
    category = 2346301904   
    conv = drop_nulls(global_conv, ['quantity'])
    items = drop_nulls(global_items, ['category'])
    users = drop_nulls(global_users, ['registerCountry'])

    joined_items_conv = pd.merge(items, conv, left_on='itemId', right_on='itemId', how='inner')
    joined = pd.merge(joined_items_conv, users, left_on='userId', right_on='userId', how='inner')

    filtered = joined[joined['category']==category]
    grouped = filtered.groupby('registerCountry')['quantity'].sum()

    ax = grouped.plot(kind='bar')
    ax.set_xlabel('Category')
    ax.set_ylabel('Quantity')
    ax.set_title("Number of items from "+str(category)+" category, grouped by country.", fontsize=18)

    plt.xticks(rotation=70)
    plt.tight_layout()
    plt.show()

Displays number of items from particular category purchased by people from all countries.

In [ ]:
def number_of_items_purchased_in_particular_country_grouped_by_category(country):
    country = 'United States'
    conv = drop_nulls(global_conv, ['quantity'])
    items = drop_nulls(global_items, ['category'])
    users = drop_nulls(global_users, ['registerCountry'])

    joined_items_conv = pd.merge(items, conv, left_on='itemId', right_on='itemId', how='inner')
    joined = pd.merge(joined_items_conv, users, left_on='userId', right_on='userId', how='inner')

    filtered = joined[joined['registerCountry']==country]
    grouped = filtered.groupby('category')['quantity'].sum()

    ax = grouped.plot(kind='bar')
    ax.set_xlabel('Category')
    ax.set_ylabel('Quantity')
    ax.set_title("Number of items purchased in " + country + ", grouped by category.", fontsize=18)

    plt.xticks(rotation=70)
    plt.tight_layout()
    plt.show()

Displays number of items from all cattegories purchased in particular country.

In [15]:
def number_of_purchased_items_grouped_by_categories_in_all_countries():
    conv = drop_nulls(global_conv, ['quantity'])
    items = drop_nulls(global_items, ['category'])
    users = drop_nulls(global_users, ['registerCountry'])

    joined_items_conv = pd.merge(items, conv, left_on='itemId', right_on='itemId', how='inner')
    joined = pd.merge(joined_items_conv, users, left_on='userId', right_on='userId', how='inner')

    y_labels = []
    x_labels = []
    y_ticks = []
    x_ticks = []

    for i, row in enumerate(joined['registerCountry'].unique()):
        y_labels.append(row)
        y_ticks.append(i)

    for i, row in enumerate(joined['category'].unique()):
        x_labels.append(row)
        x_ticks.append(i)

    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)

    for j, country in enumerate(y_labels):
        r = np.random.rand(1)
        color = cm.rainbow(np.linspace(r, r+1, 1))       
        for i, category in enumerate(x_labels):
            countryFiltered = joined[joined['registerCountry']==country]
            filtered = countryFiltered[countryFiltered['category']==category]
            area=filtered['quantity'].sum()
            ax.scatter(x=i, y=j, s=area, c=color, alpha=0.7)  

    ax.set(xticks=range(len(x_labels)), xticklabels=x_labels,
       yticks=range(len(y_labels)), yticklabels=y_labels)

    ax.set_xlabel('Category')
    ax.set_ylabel('Country')
    ax.set_title('Sum of purchased items, grouped by country and category.', fontsize=18)

    plt.tick_params(axis='x')
    plt.xticks(rotation=90)
    plt.tight_layout()

    plt.show()

Displays general view on sum of purchased items grouped by country and category.

In [12]:
def number_of_purchased_items_after_seeing_campaigns_grouped_by_categories():
    #plot generated on random 0.15 of the data
    conv = drop_nulls(global_conv, ['timestamp'])
    users_ads = drop_nulls(global_users_ads, ['utmCampaign'])
    users = drop_nulls(global_users, ['signupTime'])
    items = drop_nulls(global_items, ['category'])

    users['week_after'] = pd.DatetimeIndex(users['signupTime'])+pd.Timedelta(days=7)
    conv['timestamp'] = pd.DatetimeIndex(conv['timestamp'])

    joined = pd.merge(users, users_ads, left_on='userId', right_on='userId', how='inner')
    joined = pd.merge(joined, conv, left_on='userId', right_on='userId', how='inner')
    joined = pd.merge(joined, items, left_on='itemId', right_on='itemId', how='inner')

    filtered = joined[joined['timestamp'] <= joined['week_after']]

    y_labels = []
    x_labels = []
    y_ticks = []
    x_ticks = []
    x = []
    y = []

    for i, row in enumerate(filtered['utmCampaign'].unique()):
        y_labels.append(row)
        y_ticks.append(i)
        y.append(i)

    for i, row in enumerate(filtered['category'].unique()):
        x_labels.append(row)
        x_ticks.append(i)
        x.append(i)

    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)

    for j, utmCampaign in enumerate(y_labels):
        r = np.random.rand(1)
        color = cm.rainbow(np.linspace(r, r+1, 1))       
        for i, category in enumerate(x_labels):
            utmCampaignFiltered = filtered[filtered['utmCampaign']==utmCampaign]
            categoryFiltered = utmCampaignFiltered[utmCampaignFiltered['category']==category]
            area=categoryFiltered['quantity'].sum()
            ax.scatter(x=i, y=j, s=area, c=color, alpha=0.7)  

    ax.set(xticks=range(len(x_labels)), xticklabels=x_labels,
       yticks=range(len(y_labels)), yticklabels=y_labels)

    ax.set_xlabel('Category')
    ax.set_ylabel('utmCampaign')
    ax.set_title('Sum of purchased items in the first week after signing in, grouped by campaigns and category.', fontsize=18)

    plt.tick_params(axis='x', labelsize=8)
    plt.xticks(rotation=70)
    plt.tight_layout()

    plt.show()

Displays general view on sum of purchased items in the first week after signing in, connected with campaigns being seen, grouped by category and campaign.

In [2]:
def average_number_of_purchased_items_during_the_first_month_after_signing_in():
    conv = drop_nulls(global_conv, ['timestamp', 'quantity'])
    users = drop_nulls(global_users, ['signupTime', 'userId'])

    users['signupTime'] = pd.DatetimeIndex(users['signupTime'])
    conv['timestamp'] = pd.DatetimeIndex(conv['timestamp'])

    joined = pd.merge(users, conv, left_on='userId', right_on='userId', how='inner')
    joined['purchase_day'] = pd.to_timedelta(joined['timestamp'])-pd.to_timedelta(joined['signupTime'])

    filtered = joined[joined['purchase_day']>=pd.Timedelta(days=0)]
    filtered = filtered[filtered['purchase_day']<pd.Timedelta(days=30)]
    filtered['purchase_day'] = pd.DatetimeIndex(filtered['purchase_day']).day

    aver = filtered.groupby(['purchase_day', 'userId'])['quantity'].sum()

    summ = [0 for x in range(0,31)]
    countt = [0 for x in range(0,31)]
    average = [0 for x in range(0,31)]

    for x in aver.iteritems():
        day = x[0][0]
        quantitySum = x[1]
        summ[day] += quantitySum
        countt[day] += 1

    for y in range(0, 31):
        if countt[y] != 0:
            average[y] = summ[y]/countt[y]

    ax = sns.barplot(y=average, x=[i for i in range(0, 31)])
    ax.set_xlabel('Day after signing in')
    ax.set_ylabel('Average quantity')
    ax.set_title('Average sum of purchases (quantities) during first month after signing in.', fontsize=18)

    plt.tick_params(axis='x', labelsize=8)
    plt.tight_layout()

    plt.show()

Displays average sum of purchases during first month after signing in.

In [32]:
def histogram_number_of_purchases_per_user_during_first_week_and_month():

    users = drop_nulls(global_users, ['signupTime'])
    conv = drop_nulls(global_conv, ['timestamp', 'quantity'])

    users['signupTime'] =  pd.DatetimeIndex(users['signupTime'])
    users['week_after'] = pd.DatetimeIndex(users['signupTime'])+pd.Timedelta(days=7)
    users['month_after'] = pd.DatetimeIndex(users['signupTime'])+pd.Timedelta(days=30)
    conv['timestamp'] = pd.DatetimeIndex(conv['timestamp'])

    joined = pd.merge(users, conv, left_on='userId', right_on='userId', how='inner')
    week_filtered = joined[joined['timestamp'] <= joined['week_after']]
    month_filtered = joined[joined['timestamp'] <= joined['month_after']]
    week_grouped = week_filtered.groupby('userId')['quantity'].sum()
    month_grouped = month_filtered.groupby('userId')['quantity'].sum()

    week_values = [0] * len(joined[~joined['userId'].isin(week_filtered['userId'].unique())].groupby('userId'))
    week_values.extend(week_grouped.values)
    month_values = [0] * len(joined[~joined['userId'].isin(month_filtered['userId'].unique())].groupby('userId'))
    month_values.extend(month_grouped.values)

    FirstMonth = go.Histogram(
        x=month_values,
        opacity=0.75
    )
    FirstWeek = go.Histogram(
        x=week_values,
        opacity=0.75
    )
    data = [FirstWeek, FirstMonth]
    layout = go.Layout(
        barmode='overlay',
        title='Histogram: number of purchases per user during first week and first month after registration',
            xaxis=dict(
                title='Bins',
                titlefont=dict(
                    family='Courier New, monospace',
                    size=18,
                    color='#7f7f7f'
                )
            ),
            yaxis=dict(
                title='Quantity',
                titlefont=dict(
                    family='Courier New, monospace',
                    size=18,
                    color='#7f7f7f'
                )
            )
        )
    fig = go.Figure(data=data, layout=layout)
    plot_url = py.plot(fig, filename='overlaid-histogram')
    print(plot_url)

https://plot.ly/~PythonAPI/272


/home/gaba/anaconda3/lib/python3.5/site-packages/plotly/plotly/plotly.py:236: UserWarning:

Woah there! Look at all those points! Due to browser limitations, the Plotly SVG drawing functions have a hard time graphing more than 500k data points for line charts, or 40k points for other types of charts. Here are some suggestions:
(1) Use the `plotly.graph_objs.Scattergl` trace object to generate a WebGl graph.
(2) Trying using the image API to return an image instead of a graph URL
(3) Use matplotlib
(4) See if you can create your visualization with fewer data points

If the visualization you're using aggregates points (e.g., box plot, histogram, etc.) you can disregard this warning.



Shows distribution of how many items users have bought during first week after sign up.

In [17]:
def probability_of_purchase_during_the_first_month_after_registration():
    conv = drop_nulls(global_conv, ['timestamp', 'quantity'])
    users = drop_nulls(global_users, ['signupTime'])

    users['signupTime'] = pd.DatetimeIndex(users['signupTime'])
    conv['timestamp'] = pd.DatetimeIndex(conv['timestamp'])

    joined = pd.merge(users, conv, left_on='userId', right_on='userId', how='inner')
    joined['purchase_day'] = pd.to_timedelta(joined['timestamp'])-pd.to_timedelta(joined['signupTime'])

    nr_of_users = len(joined.groupby(['userId']))

    filtered = joined[joined['purchase_day']>=pd.Timedelta(days=0)]
    filtered = filtered[filtered['purchase_day']<pd.Timedelta(days=30)]
    filtered['purchase_day'] = pd.DatetimeIndex(filtered['purchase_day']).day

    purch = filtered.groupby(['purchase_day'])['userId'].count()
    probabil = [0 for x in range(0,31)]

    for x in purch.iteritems():
        day = x[0]
        probabil[day] = x[1]/nr_of_users

    ax = sns.barplot(y=probabil, x=[i for i in range(0, 31)])
    ax.set_xlabel('Day after registration')
    ax.set_ylabel('Probability (sum of purchases/number of users who have at least one purchase)')
    ax.set_title('Probability of purchase during the first month after registration', fontsize=18)

    plt.tick_params(axis='x', labelsize=8)
    plt.tight_layout()

    plt.show()

/home/gaba/anaconda3/lib/python3.5/site-packages/numpy/lib/arraysetops.py:379: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison

/home/gaba/anaconda3/lib/python3.5/site-packages/matplotlib/__init__.py:892: UserWarning:

axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.



Shows how probable is user's purchase during the first month after sign up.

In [106]:
def to_percent(y, position):
    s = str(y)
    if matplotlib.rcParams['text.usetex'] is True:
        return s + r'$\%$'
    else:
        return s + '%'

In [60]:
def h_pd_igd_weekly_user_count_of_purchases():
    conv = drop_nulls(global_conv, ['timestamp'])

    conv['week'] = pd.DatetimeIndex(conv['timestamp']).week
    conv['year'] = pd.DatetimeIndex(conv['timestamp']).year

    grouped = conv.groupby(['year', 'week'])['quantity'].sum()
    aver = math.floor(grouped.mean())

    purch_sum = sum(grouped.values)
    mu = math.floor(grouped.mean())

    counts = poisson.rvs(mu, size=100)
    bins = range(0, max(counts)+2)
    bins = bins[0::10]

    #Histogram
    weights=100*np.ones_like(grouped.values)/len(grouped.values)
    ax = plt.hist(grouped.values, bins=bins, align="left", histtype="step", color="black", label='Histogram of the data') 

    #Poisson Distribution
    x = range(0,2*max(counts))
    prob = poisson.pmf(x, mu)*100
    plt.plot(x, prob, label='Poisson Distribution')

    #Inverse Gamma Distribution
    y = grouped.values
    mle = math.floor(np.mean(y))
    rv = invgamma(1)

    x = np.linspace(-10, np.minimum(rv.dist.b, max(grouped.values)))
    h = plt.plot(x, rv.pdf(x)*100000, label='Inverse Gamma Distribution')

    plt.tick_params(axis='x', labelsize=8)
    plt.legend(bbox_to_anchor=(0.7, 0.9), loc=1, borderaxespad=0.)
    plt.title('Poisson Distribution of weekly users\' count of purchases', fontsize=18)

    ax = plt.gca()

    plt.tight_layout()
    plt.show()

range(0, 3088)
range(0, 3088, 10)


Shows histogram, Poisson Distribution and Inverse Gamma Distribution of weekly users' count of purchases.

In [ ]:
#def number_of_active_users_per_month_and_number_of_registered():
#number of active users per month
users = global_users
conv = drop_nulls(global_conv, ['timestamp'])

conv['month'] = pd.DatetimeIndex(conv['timestamp']).month
conv['year'] = pd.DatetimeIndex(conv['timestamp']).year

joined = pd.merge(users, conv, left_on='userId', right_on='userId', how='inner')
grouped = joined.groupby(['year', 'month'])['userId'].unique()

x_labels = []
x_ticks = []
for i, item in enumerate(grouped.iteritems()):
    if(item[0][0] == 0):#degenerated data (0th month)
        continue
    x_labels.append(str(item[0][0])+" / "+str(item[0][1]))
    x_ticks.append(i)

y = []
for item in grouped.iteritems():
    y.append(len(item[1]))

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.set(xticks=range(len(x_labels)), xticklabels=x_labels)

plt.plot(x_ticks, y)

#overall number of registered users per month
users = drop_nulls(global_users, ['signupTime', 'userId'])

users['month'] = pd.DatetimeIndex(users['signupTime']).month
users['year'] = pd.DatetimeIndex(users['signupTime']).year

grouped = users.groupby(['year', 'month'])['userId'].count()
sortedd = grouped.sort_index()

for i, x in enumerate(grouped.iteritems()):
    print(str(i)+" "+str(x))

y =[]
for i, x in enumerate(grouped):
    if i != 0:
        y.append(x+y[i-1])
    else:
        y.append(x)

plt.plot(x_ticks, y, kind='bar')

ax = plt.gca()
ax.set_xlabel('Year and month')
ax.set_ylabel('Quantity')
ax.set_title('Number of active users per month and number of registered', fontsize=18)

plt.xticks(rotation=70)
plt.tight_layout()
plt.show()

Number of users who have at least one purchase in particular month.

In [8]:
#def variance_sum_of_revenue_per_user_in_each_month():
users = drop_nulls(global_users, ['userId'])
conv = drop_nulls(global_conv, ['timestamp'])

conv['conv_month'] = pd.DatetimeIndex(conv['timestamp']).month
conv['conv_year'] = pd.DatetimeIndex(conv['timestamp']).year

users['signup_month'] = pd.DatetimeIndex(users['signupTime']).month
users['signup_year'] = pd.DatetimeIndex(users['signupTime']).year

joined = pd.merge(users, conv, left_on='userId', right_on='userId', how='inner')
grouped = joined.groupby(['conv_year', 'conv_month', 'userId'])['price'].sum()
grouped = grouped.sort_index()
signup_grouped = users.groupby(['signup_year', 'signup_month'])['userId'].count()
signup_grouped = signup_grouped.sort_index()
for i, x in enumerate(signup_grouped):
    if i != 0:
        signup_grouped[i] += signup_grouped[i-1]

signed = []
for i in range(0, 3):
    signed.append([])
    for j in range(0, 13):
        signed[i].append(0)

for i, year_id in enumerate(signup_grouped.index.labels[0]):
    month_id = signup_grouped.index.labels[1][i]
    year = signup_grouped.index.levels[0][year_id]
    month = signup_grouped.index.levels[1][month_id]
    signed[year-2013][month] = signup_grouped[i]

res = []
for i in range(0, 3):
    res.append([])
    for j in range(0, 13):
        res[i].append(0)

#count variance for each month
m = -1
y = -1
data = []
values = grouped.values
for i, year_id in enumerate(grouped.index.labels[0]):
    month_id = grouped.index.labels[1][i]
    year = grouped.index.levels[0][year_id]
    month = grouped.index.levels[1][month_id]
    if(i != 0 and (y != year or m != month)) or (i == len(grouped.index.labels[0])-1):
        if i == len(grouped.index.labels[0])-1:
            data.append(values[i])
        if signed[y-2013][m] > 0:
            if i == len(grouped.index.labels[0])-1:
                y = year
                m = month
            data.extend([0] * (signed[y-2013][m] - len(data)))
            res[y-2013][m] = statistics.variance(data)
        data = []
    data.append(values[i])
    y = year
    m = month

variance = []
labels = []                                                 
for i in range(0, 3):
    for j in range(0, 12):
        variance.append(res[i][j+1])
        labels.append(str(j+1)+" / "+str(i+2013))

ax = sns.barplot(y=variance, x=labels)
ax.set_xlabel('Year and month')
ax.set_ylabel('Variance')
ax.set_title('Variance: sum of prices of purchases for each user in each month after sign up', fontsize=18)

plt.tick_params(axis='x', labelsize=10)
plt.xticks(rotation=70)
plt.tight_layout()

plt.show()

/home/ducki13/anaconda3/lib/python3.5/site-packages/numpy/lib/arraysetops.py:379: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



Shows variance in revenue among active users in each month.

In [25]:
def stanDev(values):
    length = len(values)
    m = statistics.mean(values)
    total_sum = 0
    for i in range(length):
        total_sum += (values[i]-m)**2
 
    return math.sqrt(total_sum/length)

In [26]:
def normDistr(x, m, stDev):
    return (math.e**((-((x-m)**2))/(2*stDev)))/(math.sqrt(stDev)*math.sqrt(2*math.pi))

In [34]:
def normal_distribution_weekly_revenue:
    conv = drop_nulls(global_conv, ['timestamp'])

    conv['year'] = pd.DatetimeIndex(conv['timestamp']).year
    conv['week'] = pd.DatetimeIndex(conv['timestamp']).week

    grouped = conv.groupby(['year', 'week'])['price'].sum()

    _mean = statistics.mean(grouped.values)
    _stDev = stanDev(grouped.values)

    x = np.linspace(0,max(grouped.values)+100,100)
    y = norm.pdf(x, loc=_mean, scale=_stDev)

    pylab.plot(x,y)
    plt.title("Normal Distribution of Weekly Revenue")
    pylab.show()

Shows normal distribution of weekly revenue.